In [60]:
import torch

In [230]:
T = torch.tensor([
    # s = 0, a = 0,1,2
    [
        [0.5, 0.5],
        [0.5, 0.5],
        [1.0, 0.0],
    ],
    # s = 1, a = 0,1,2
    [
        [0.5, 0.5],
        [0.5, 0.5],
        [0.0, 1.0],
    ]
])

p = 0.85
O = torch.tensor([
    # a = 0, s' = 0,1
    [
        [0.5, 0.5],
        [0.5, 0.5],
    ],
    # a = 1, s' = 0,1
    [
        [0.5, 0.5],
        [0.5, 0.5],
    ],
    # a = 2, s' = 0,1
    [
        [p, 1-p],
        [1-p, p],
    ]
])

R = torch.tensor([
    # s = 0
    [+10, -100, -1],
    # s = 1
    [-100, +10, -1],
]).float()

In [349]:
opt_fsc_action = torch.tensor([2, 2, 0, 2, 1])
opt_fsc_state = torch.tensor([
    [1, 3],
    [2, 0],
    [0, 0],
    [0, 4],
    [0, 0],
])

In [350]:
gamma = 0.95

num_nodes, num_states = len(fsc_action), T.shape[0]
num_obs = O.shape[2]

V = torch.zeros((num_nodes, num_states))
for idx in range(200):
    prev = torch.clone(V)
    for n in range(num_nodes):
        for s in range(num_states):
            a = fsc_action[n]
            V[n, s] = R[s, a] + gamma * sum([
                T[s, a, ns] * sum(
                    O[a, ns, o] * V[fsc_state[n, o], ns]
                    for o in range(num_obs)
                )
                for ns in range(num_states)
            ])
    if torch.norm(prev-V) < 1e-5:
        print('Converged', idx)
        break
print(V)

IndexError: tensors used as indices must be long, byte or bool tensors

In [351]:
def policy_evaluation(fsc_action, fsc_state, T, O, R, *, gamma=0.95, debug=True, max_iter=5000, vi_eps=1e-3):
    num_nodes, num_states = len(fsc_action), T.shape[0]
    num_obs = O.shape[2]

    V = torch.zeros((num_nodes, num_states))
    for idx in range(max_iter):
        prev = torch.clone(V)
        for n in range(num_nodes):
            a = fsc_action[n]
            state_vals = torch.sum(O[a] * V[fsc_state[n]].T, axis=1)
            V[n] = R[:, a] + gamma * T[:, a] @ state_vals
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                a = fsc_action[n]
                nn = fsc_state[n]

                state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
                V[n, s] = R[s, a] + gamma * T[s, a] @ state_vals
        '''
        if torch.norm(prev-V) < vi_eps:
            if debug:
                print('Converged', idx)
            break
    return V
policy_evaluation(opt_fsc_action, opt_fsc_state, T, O, R)

Converged 108


tensor([[ 19.3677,  19.3677],
        [ 24.6921,   3.0113],
        [ 28.3993, -81.6007],
        [  3.0113,  24.6921],
        [-81.6007,  28.3993]])

# Stochastic controllers...

In [234]:
def det_to_stoch(fsc_action, fsc_state, T):
    num_states, num_actions, _ = T.shape
    num_nodes, num_obs = fsc_state.shape
    a = torch.zeros((num_nodes, num_actions))
    s = torch.zeros((num_nodes, num_actions, num_obs, num_nodes))
    for n in range(num_nodes):
        a[n, fsc_action[n]] = 1.
        for o in range(num_obs):
            s[n, fsc_action[n], o, fsc_state[n, o]] = 1.
    return a, s

det_to_stoch(fsc_action, fsc_state, T)

(tensor([[0., 0., 1.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.]]), tensor([[[[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 1., 0., 0., 0.],
           [0., 0., 0., 1., 0.]]],
 
 
         [[[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 0., 1., 0., 0.],
           [1., 0., 0., 0., 0.]]],
 
 
         [[[1., 0., 0., 0., 0.],
           [1., 0., 0., 0., 0.]],
 
          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]]],
 
 
         [[[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],
 
          [[1., 0., 0., 0., 0.],
           [0., 0., 0., 0., 1.]]],
 
 
         [[[0., 0., 0., 0., 0.],
           

In [352]:
def policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, *, gamma=0.95, debug=True, max_iter=5000, vi_eps=1e-3):
    num_states, num_actions, _ = T.shape
    num_nodes, num_obs = fsc_state.shape[0], O.shape[2]

    V = torch.zeros((num_nodes, num_states))
    for idx in range(max_iter):
        prev = torch.clone(V)
        for n in range(num_nodes):
            # (a, s')
            val = torch.sum(
                # (a, s', o) * (a, None, o, n') = (a, s', o, n') -(sum(ax=2))> (a, s', n')
                (O[:, :, :, None] * fsc_state[n, :, None]).sum(axis=2) *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = torch.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n] # HAKKK 
            #V[n] = R @ fsc_action[n] + gamma * Q.sum(1) # HAKKK  # HAKKK  # HAKKK 
        '''
        for n in range(num_nodes):
            # (a, s')
            val = torch.sum(
                # (a, s', o) @ (o, n') = (a, s', n')
                O @ fsc_state[n] *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = torch.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n]
        '''
        '''
        for n in range(num_nodes):
            # (a, s')
            val = np.sum(
                # (a, s', o) @ (o, n') = (a, s', n')
                O @ fsc_state[n] *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = np.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n]
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                # (a, s')
                val = np.sum(
                    # (a, s', o) @ (o, n') = (a, s', n')
                    O @ fsc_state[n] *
                    V.T[None, :, :], axis=2)
                # (a) expected future value for actions
                val = np.sum(T[s] * val, axis=1)
                V[n, s] = fsc_action[n] @ (R[s] + gamma * val)
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                V[n, s] = sum(
                    fsc_action[n, a] * (
                        R[s, a] + gamma * sum([
                            T[s, a, ns] * O[a, ns, :] @ fsc_state[n, :, :] @ V[:, ns]
                            for ns in range(num_states)
                        ])
                    )
                    for a in range(num_actions))
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                V[n, s] = sum(fsc_action[n, a] * (R[s, a] + gamma * sum([
                    T[s, a, ns] * sum(
                        O[a, ns, o] * sum(fsc_state[n, o, nn] * V[nn, ns] for nn in range(num_nodes))
                        for o in range(num_obs)
                    )
                    for ns in range(num_states)
                ])) for a in range(num_actions))
        '''
        '''
        for n in range(num_nodes):
            a = fsc_action[n]
            state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
            V[n] = R[:, a] + gamma * T[:, a] @ state_vals
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                a = fsc_action[n]
                nn = fsc_state[n]

                state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
                V[n, s] = R[s, a] + gamma * T[s, a] @ state_vals
        '''
        if torch.norm(prev-V) < vi_eps:
            if debug:
                print('Converged', idx)
            break
    return V

# TODO, make SR !@ TODO TODO
V = policy_evaluation_stoch(*det_to_stoch(opt_fsc_action, opt_fsc_state, T) + (T, O, R))
print(V.shape, V)

Converged 108
torch.Size([5, 2]) tensor([[ 19.3677,  19.3677],
        [ 24.6921,   3.0113],
        [ 28.3993, -81.6007],
        [  3.0113,  24.6921],
        [-81.6007,  28.3993]])


In [236]:
# trying to write the constraint out...
#ret = (T * (O * (canz@V).transpose(1, 2)).sum(2)[None, :, :]).sum(axis=(1, 2))
#R@c_a + gamma * ret

In [359]:
def make_constraints(n, V, T, O, R):
    num_states, num_actions, _ = T.shape
    num_nodes = V.shape[0]
    num_obs = O.shape[2]

    canz_shape = torch.Size((num_actions, num_obs, num_nodes))

    args = 1 + num_actions + canz_shape.numel()

    delta_slice = slice(0, 1)
    c_a_slice = slice(1, 1 + num_actions)
    canz_slice = slice(1 + num_actions, None)

    G = torch.zeros((V.shape[1] + num_actions + canz_shape.numel(), args))
    h = torch.cat((
        -V[n],
        torch.zeros(num_actions + canz_shape.numel()),
    ))

    # Last spots encode c >= 0 as -c <= 0
    for idx in range(num_actions + canz_shape.numel()):
        G[num_states+idx, delta_slice.stop+idx] = -1

    for s in range(num_states):
        G[s, delta_slice] = 1
        G[s, c_a_slice] = -R[s]
        canz_coef = G[s, canz_slice].view(canz_shape)

        # (a, s', o)
        #xx = T[s, :, :, None]*O
        for a in range(num_actions):
            for o in range(num_obs):
                for nn in range(num_nodes):
                    canz_coef[a, o, nn] = -gamma * (T[s, a, :] * O[a, :, o] * V[nn]).sum()

    # Q has to be PSD... so I've scaled it down considerably here
    Q = torch.eye(args) / 10000.
    p = torch.zeros(args)
    # The default optimization is an argmin, so this lets us maximize the delta!
    p[delta_slice] = -1

    # HACK HACK in the future, write these so we don't need c_a?? to be simpler??
    A = torch.zeros((num_actions+1, args))
    b = torch.zeros(A.shape[0])

    # \sum_a c_a = 1
    A[num_actions, c_a_slice] = 1
    b[num_actions] = 1

    # for each a, \sum_a c_a_n_z = c_a
    for a in range(num_actions):
        A[a, c_a_slice][a] = -1
        A[a, canz_slice].view(canz_shape)[a] = 1

    return Q, p, G, h, A, b

Q, p, G, h, A, b = make_constraints(0, V, T, O, R)
#A, b
G[2], h[2]

(tensor([ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.]), tensor(0.))

In [360]:
import qpth
constraints = make_constraints(0, V, T, O, R)
soln = qpth.qp.QPFunction(verbose=-1)(*constraints).float()


In [361]:
soln.flatten()[canz_slice].view(canz_shape).sum()

tensor(0.9897)

In [362]:
[
    soln.flatten()[delta_slice],
    soln.flatten()[c_a_slice],
    soln.flatten()[canz_slice].view(canz_shape),
]

[tensor([0.1654]),
 tensor([-8.2436e-04,  4.4796e-03,  1.0441e+00]),
 tensor([[[0.0004, 0.0010, 0.0008, 0.0009, 0.0009],
          [0.0005, 0.0010, 0.0009, 0.0005, 0.0009]],
 
         [[0.0022, 0.0047, 0.0023, 0.0051, 0.0034],
          [0.0027, 0.0042, 0.0031, 0.0052, 0.0030]],
 
         [[0.1067, 0.1176, 0.0969, 0.1147, 0.0929],
          [0.0887, 0.0882, 0.0791, 0.0697, 0.0916]]])]

In [357]:
fsc_action[n], fsc_state[n], soln.flatten()[canz_slice].view(canz.shape)[0, 1].sum()

(tensor([0.3333, 0.3333, 0.3333]),
 tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]],
 
         [[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]],
 
         [[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]]]),
 tensor(-1.8173e-08))

# Trying to fit...

In [371]:
# canz -> p(n' | n, a, o) -> p(n', a, o | n) ??
# c_a -> p(a | n)
fsc_action = torch.ones((num_nodes, num_actions))/num_actions
fsc_state = torch.ones((num_nodes, num_actions, num_obs, num_nodes))/(num_actions+num_obs+num_nodes)

#fsc_action = torch.tensor(np.random.uniform(0, 1, size=(num_nodes, num_actions))).softmax(1).float()
#xrnd = np.random.uniform(0, 1, size=(num_nodes, num_actions, num_obs, num_nodes))
#fsc_state = torch.tensor(xrnd/np.sum(xrnd, axis=(1, 2, 3))).float()

V = policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
for i in range(3):
    prev = V.clone()
    for n in range(num_nodes):
        V = policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
        constraints = make_constraints(n, V, T, O, R)
        soln = qpth.qp.QPFunction(verbose=-1)(*constraints).float()
        delta = soln[0, 0].item()
        print(n, delta)
        print([
            soln.flatten()[delta_slice],
            soln.flatten()[c_a_slice],
            #soln.flatten()[canz_slice].view(canz_shape),
        ])
        print([
            soln.flatten()[delta_slice],
            soln.flatten()[c_a_slice].sum(),
            soln.flatten()[canz_slice].view(canz_shape).sum(),
        ])
        if delta > 0:
            fsc_action[n] = soln[0, c_a_slice]
            fsc_state[n] = soln[0, canz_slice].view(canz_shape)
        print(V[0], (prev-V).sum())

0 -35.173397064208984
[tensor([-35.1734]), tensor([0.6718, 0.7113, 0.0646])]
[tensor([-35.1734]), tensor(1.4477), tensor(0.3652)]
tensor([-57.7777, -57.7777]) tensor(0.)
1 -35.17386245727539
[tensor([-35.1739]), tensor([0.6440, 0.6359, 0.0031])]
[tensor([-35.1739]), tensor(1.2830), tensor(0.2209)]
tensor([-57.7777, -57.7777]) tensor(0.)
2 17.074920654296875
[tensor([17.0749]), tensor([0.2497, 0.0278, 0.7399])]
[tensor([17.0749]), tensor(1.0174), tensor(1.0579)]
tensor([-57.7777, -57.7777]) tensor(0.)
3 39.819095611572266
[tensor([39.8191]), tensor([ 3.7605e-10, -1.0252e-07,  1.0000e+00])]
[tensor([39.8191]), tensor(1.), tensor(1.0000)]
tensor([-52.8382, -53.2629]) tensor(-99.5180)
4 40.709922790527344
[tensor([40.7099]), tensor([-3.9144e-07,  1.9165e-07,  1.0000e+00])]
[tensor([40.7099]), tensor(1.0000), tensor(1.0000)]
tensor([-46.6384, -47.2931]) tensor(-227.6203)
0 37.90904998779297
[tensor([37.9090]), tensor([3.0251e-08, 3.8962e-08, 1.0000e+00])]
[tensor([37.9090]), tensor(1.), ten

In [372]:
prev = -1
for idx in zip(*torch.where(fsc_state>.01)):
    if prev != idx[0]:
        print('action', [(i, p) for i, p in enumerate(fsc_action[idx[0]].numpy()) if p > 1e-2])
    print(idx, fsc_state[idx])
    prev = idx[0]

action [(2, 0.99182117)]
(tensor(0), tensor(2), tensor(0), tensor(3)) tensor(0.9911)
action [(2, 0.99974215)]
(tensor(1), tensor(2), tensor(0), tensor(3)) tensor(0.9997)
action [(0, 0.18792835), (2, 0.8120717)]
(tensor(2), tensor(0), tensor(0), tensor(1)) tensor(0.0940)
(tensor(2), tensor(0), tensor(1), tensor(1)) tensor(0.0939)
(tensor(2), tensor(2), tensor(0), tensor(2)) tensor(0.8121)
action [(0, 0.037493937), (2, 0.9705136)]
(tensor(3), tensor(2), tensor(0), tensor(2)) tensor(0.9699)
action [(2, 1.0000001)]
(tensor(4), tensor(2), tensor(0), tensor(3)) tensor(0.5534)
(tensor(4), tensor(2), tensor(1), tensor(3)) tensor(0.4466)


[(2, 1.0)]